# API calls with the `requests` package

This notebook introduces the `requests` package, which allows you to perform HTTP requests from within Python.
It's more convenient than using `curl` from the command line, at least for me.

Run the API server (`03_get-post-put-delete.py`) in another console window,
then execute this script line by line to access it.

Note: You don't need to be in the virtualenv here, because you only need the
requests package and are communicating via HTTP.

Adapted from https://realpython.com/python-requests/

In [1]:
import requests

In [2]:
# This results in a 404:
requests.get('http://localhost:5003/something')

<Response [404]>

In [3]:
# This results in a 200:
requests.get('http://localhost:5003/user/Carol')

<Response [200]>

## Inspect the success/fail status

In [4]:
response = requests.get('http://localhost:5003/user/Carol')
response.status_code

200

In [5]:
bool(response)
# 'response' is cast to False if its status_code is from 400 to 599,
# and True otherwise

True

In [6]:
# This does nothing for successful requests, but raises an error
# if the HTTP status is from 400 to 599:
response.raise_for_status()

## Inspect the content

In [7]:
response.content  # raw bytes

b'{\n    "name": "Carol",\n    "age": 45,\n    "job": "Types stuff"\n}\n'

In [8]:
response.text  # string

'{\n    "name": "Carol",\n    "age": 45,\n    "job": "Types stuff"\n}\n'

In [9]:
response.json()  # creates a dict from the string

{'name': 'Carol', 'age': 45, 'job': 'Types stuff'}

## Customizing GET requests

If you want to append parameters programmatically, use the `params` argument to `get()`.
The following call is equivalent to the URL `https://api.github.com/search/repositories?q=requests&language=python`:

In [10]:
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests', 'language': 'python'}
)

In [11]:
response.json()['items'][0]['full_name']

'kennethreitz/requests'

## POST, PUT, DELETE, etc.

In [12]:
requests.post(
    'http://localhost:5003/user/NewGuy', 
    data={
        'age': 123,
        'job': 'yes'
    }
)

<Response [201]>

In [13]:
requests.get('http://localhost:5003/user/NewGuy').json()

{'name': 'NewGuy', 'age': '123', 'job': 'yes'}

Happy Birthday, NewGuy! You're fired!

In [14]:
requests.put(
    'http://localhost:5003/user/NewGuy', 
    data={
        'age': 124,
        'job': 'no'
    }
)

<Response [200]>

In [15]:
requests.get('http://localhost:5003/user/NewGuy').json()

{'name': 'NewGuy', 'age': '124', 'job': 'no'}

Now delete the old bum:

In [16]:
requests.delete('http://localhost:5003/user/NewGuy')

<Response [200]>

In [17]:
requests.get('http://localhost:5003/user/NewGuy').json()

'User not found!'

There is also HEAD, PATCH, and OPTIONS, but I don't know yet what they do.

## Authentication

Calling the /user/ endpoint results in a "HTTP 401 - Unauthorized" error:

In [18]:
resp = requests.get('https://api.github.com/user')

resp.status_code, resp.text

(401,
 '{"message":"Requires authentication","documentation_url":"https://developer.github.com/v3/users/#get-the-authenticated-user"}')

You must pass the `auth` argument to `get()`, and enter your password. Note that now it's crucial that you
use HTTPS instead of HTTP:

In [19]:
from getpass import getpass

response = requests.get('https://api.github.com/user', auth=('AlexEngelhardt', getpass()))

········


In [20]:
response.json()['name']

'Alexander Engelhardt'

There are better ways to authenticate, but this one is the basic version that works.

## Timeouts

In [21]:
from requests.exceptions import Timeout

try:
    response = requests.get('https://api.github.com', timeout=1)
except Timeout:
    print('The request timed out')
else:
    print('The request did not time out')

The request did not time out


## Sessions

meh :)